In [40]:
import tensorflow as tf
import os
import numpy as np

In [41]:
base_dir=r"D:\university\Thesis\final\Flower kath golap\1training"

In [42]:
IMAGE_SIZE=224
BATCH_SIZE=64

train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    
    
    rescale=1/255,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1)

validation_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    
    validation_split=0.1
)
    
    

In [43]:
train_genarator=train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
    

) 

validation_generator=validation_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
    

) 

Found 1054 images belonging to 3 classes.
Found 116 images belonging to 3 classes.


In [44]:
from tensorflow.keras.layers import Input,Flatten,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from glob import glob

In [45]:
IMAGE_SIZE=[224,224]
vgg=VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)
vgg.output


<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [46]:
for layer in vgg.layers:
    layer.trainable=False
    

In [47]:
folders=glob(r"D:\university\Thesis\final\Flower kath golap\1training\*")
print(len(folders))

3


In [48]:
x=Flatten()(vgg.output)
prediction=Dense(len(folders),activation='softmax')(x)
model=Model(inputs=vgg.input,outputs=prediction)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [49]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [50]:
epoch=10

history=model.fit(train_genarator,
                  steps_per_epoch=len(train_genarator),
                  epochs=epoch,
                  validation_data=validation_generator,
                  validation_steps=len(validation_generator)
                 )

Epoch 1/10
17/17 [==============================] - 249s 14s/step - loss: 0.7163 - accuracy: 0.4459 - val_loss: 0.6280 - val_accuracy: 0.5086
Epoch 2/10
17/17 [==============================] - 233s 14s/step - loss: 0.5545 - accuracy: 0.5731 - val_loss: 0.6029 - val_accuracy: 0.5690
Epoch 3/10
17/17 [==============================] - 227s 13s/step - loss: 0.5040 - accuracy: 0.6395 - val_loss: 0.6098 - val_accuracy: 0.5862
Epoch 4/10
17/17 [==============================] - 162s 9s/step - loss: 0.4702 - accuracy: 0.6736 - val_loss: 0.5756 - val_accuracy: 0.6034
Epoch 5/10
17/17 [==============================] - 105s 6s/step - loss: 0.4392 - accuracy: 0.7135 - val_loss: 0.5775 - val_accuracy: 0.5948
Epoch 6/10
17/17 [==============================] - 109s 6s/step - loss: 0.4154 - accuracy: 0.7315 - val_loss: 0.6106 - val_accuracy: 0.5517
Epoch 7/10
17/17 [==============================] - 113s 7s/step - loss: 0.3980 - accuracy: 0.7505 - val_loss: 0.5951 - val_accuracy: 0.5776
Epoch 8/10

In [68]:
from keras.preprocessing import image
import numpy as np
import keras

img_pred=keras.utils.load_img(r"D:\university\Thesis\final\Flower kath golap\2testing\Pre Bloom\E0A4OHJK7TUS.jpg",target_size=(224,224))
img_pred=tf.keras.preprocessing.image.img_to_array(img_pred)
img_pred=np.expand_dims(img_pred, axis=0)
img_pred=img_pred/255

rslt= model.predict(img_pred)

print(rslt)
if rslt[0][0]>rslt[0][1] and rslt[0][0]>rslt[0][2]:
    prediction="Bloom state"
elif rslt[0][0]<rslt[0][1] and rslt[0][1]>rslt[0][2]:
    prediction="Post Bloom state"
elif rslt[0][0]<rslt[0][2] and rslt[0][1]<rslt[0][2]:
    prediction="Pre Bloom state"
print(prediction)

1/1 [==============================] - 0s 154ms/step
[[0.04624619 0.02891204 0.92484176]]
Pre Bloom state
